This notebook demonstrates the Finite-Interval Forecasting Engine (FIFE).

FIFE is a Python package developed by the Institute for Defense Analyses in a research partnership with the U.S. Office of the Undersecretary of Defense for Personnel and Readiness.

FIFE applies state-of-the-art forecasting methods to [panel data](https://en.wikipedia.org/wiki/Panel_data). A panel dataset contains periodic observations of subjects. In a given period, subjects may enter or exit the panel; subjects that do not exit survive to be observed in later periods. FIFE can forecast subject-specific probabilities of survival, future states of survival, and circumstances of exit for each future period up to a given or inferred maximum time horizon.

In this example we forecast when each leader of the world's countries will lose power. To skip past how FIFE works to what FIFE can produce, see the [Forecasts section](#Forecasts).

There are many aspects of FIFE that this example does not cover; you can read about those aspects in [FIFE's documentation](https://fife.readthedocs.io/en/latest/).

We start by importing the objects we'll be using. Additionally, we include a command to suppress warning output. Note that the outputs of several cells were suppressed for the sake of readability.

In [1]:
from fife.lgb_modelers import LGBSurvivalModeler
from fife.processors import PanelDataProcessor
from fife.utils import make_results_reproducible
from pandas import concat, date_range, read_csv, to_datetime

import warnings
warnings.filterwarnings('ignore')

The following makes sure we get the same results each time we execute this code from top to bottom.

In [2]:
SEED = 9999
make_results_reproducible(SEED)

We use the July 2020 edition of the [Rulers, Elections, and Irregular Governance dataset (REIGN)](https://oefdatascience.github.io/REIGN.github.io/), a monthly panel of national leaders and political conditions since January 1950. We load the REIGN data directly from its online archive. If you plan to execute this notebook repeatedly, please avoid spamming the REIGN archive with requests by saving the data locally, then changing the file path in the cell below to your local path.

In [3]:
data = read_csv("https://www.dl.dropboxusercontent.com/s/3tdswu2jfgwp4xw/REIGN_2020_7.csv?dl=0")
data.head()

,ccode,country,leader,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,2.0,USA,Truman,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,2.0,USA,Truman,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,2.0,USA,Truman,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,2.0,USA,Truman,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,2.0,USA,Truman,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


The data need only minimal pre-processing to be ready for FIFE. Panel data are defined by two identifiers: individual and time. In the REIGN data, we can identify an individual by their name and the country they lead. We can identify time by year and month. FIFE will automatically infer the individual and time identifiers as the two left-most columns in the data, so we rearrange the columns accordingly. Along the way we drop the redundant columns `ccode` (a numeric code for country) and `leader`.

In [4]:
data["country-leader"] = data["country"] + ": " + data["leader"]
data["year-month"] = data["year"].astype(int).astype(str) + data["month"].astype(int).astype(str).str.zfill(2)
data["year-month"] = to_datetime(data["year-month"], format="%Y%m")
data = concat([data[["country-leader", "year-month"]],
               data.drop(["ccode", "country-leader", "leader", "year-month"],
                         axis=1)],
               axis=1)
data.head()

,country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,USA: Truman,1950-01-01,USA,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,USA: Truman,1950-02-01,USA,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,USA: Truman,1950-03-01,USA,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,USA: Truman,1950-04-01,USA,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,USA: Truman,1950-05-01,USA,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


By definition, each unique pair of identifier values identifies a single observation. REIGN contains a few violations of this definition, due to leaders losing and regaining power in a given month. We keep only the first observation of a given leader in a given month, thereby ignoring intra-month lapses in power.

In [5]:
total_obs = len(data)
data = data.drop_duplicates(["country-leader", "year-month"], keep="first")
n_duplicates = total_obs - len(data)
print(f"{n_duplicates} observations with a duplicated identifier pair deleted.")

7 observations with a duplicated identifier pair deleted.


FIFE's PanelDataProcessor does the rest of our data processing for us, including identifying which features are categorical, computing which leadership spells ended and when, and reserving a random 25% of leaders for model validation.

In [6]:
data_processor = PanelDataProcessor(data=data)
data_processor.build_processed_data()
data_processor.data.head()

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


,country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,...,precip,couprisk,pctile_risk,_period,_predict_obs,_test,_validation,_maximum_lead,_duration,_event_observed
0,Afghanistan: Abdallah Yakta,1967-10-01,Afghanistan,1967,10,0,53,1,0,1,...,0.018704,NaN,NaN,213,False,False,False,633,1,True
1,Afghanistan: Abdallah Yakta,1967-11-01,Afghanistan,1967,11,0,53,1,0,2,...,0.179240,NaN,NaN,214,False,False,False,632,0,True
2,Afghanistan: Abdul Zahir,1971-06-01,Afghanistan,1971,6,0,61,1,0,1,...,-1.806554,NaN,NaN,257,False,False,False,589,18,True
3,Afghanistan: Abdul Zahir,1971-07-01,Afghanistan,1971,7,0,61,1,0,2,...,-1.797812,NaN,NaN,258,False,False,False,588,17,True
4,Afghanistan: Abdul Zahir,1971-08-01,Afghanistan,1971,8,0,61,1,0,3,...,-1.815134,NaN,NaN,259,False,False,False,587,16,True


We pass the processed data to FIFE's LGBSurvivalModeler, which trains gradient-boosted tree models using [LightGBM](https://lightgbm.readthedocs.io/en/latest/). FIFE trains one model for each time horizon up to some maximum. In the absence of a specified maximum, FIFE will train up to the time horizon for which there are no more than 64 training observations that survived that long.

By default, FIFE uses [Dask](https://dask.org/) to parallelize training across time horizons, which can speed up the modeling process. However, parallelization can cause an [IOStream issue endemic to Jupyter notebooks](https://github.com/ipython/ipykernel/issues/334). We can tell FIFE to train one model at a time by specifying `parallelize=False`. You can remove the "%%capture" command in this chunk to view the function output (as is also true for any other chunk with the command), which in this case is the log of the modeling process in logical order. Each line reports the model loss on the validation set after training a new tree. Training stops once the validation loss hasn't improved for four trees in a row. Then FIFE starts training trees for the next time horizon, until it reaches the maximum.

In [7]:
%%capture

modeler = LGBSurvivalModeler(data=data_processor.data);
modeler.build_model(parallelize=False);

#### Forecasts
Obtaining a forecast for every current leader is as simple as `modeler.forecast()`. We can also customize the column headers of the forecasts. Each value represents the probability that the given leader will still be in power at the start of the given month.

In [8]:
forecasts = modeler.forecast()
custom_forecast_headers = date_range(data["year-month"].max(),
                                     periods=len(forecasts.columns) + 1,
                                     freq="M").strftime("%b %Y")[1:]
forecasts.columns = custom_forecast_headers
forecasts

,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,...,Dec 2039,Jan 2040,Feb 2040,Mar 2040,Apr 2040,May 2040,Jun 2040,Jul 2040,Aug 2040,Sep 2040
country-leader,,,,,,,,,,,,,,,,,,,,,
Afghanistan: Ashraf Ghani,0.993990,0.970821,0.946974,0.925935,0.912296,0.903354,0.898115,0.883586,0.876560,0.867222,...,0.016121,0.015504,0.015401,0.015211,0.013763,0.013671,0.013586,0.013509,0.013416,0.013333
Albania: Rama,0.994358,0.989848,0.986320,0.982195,0.974887,0.968206,0.960713,0.952167,0.941558,0.920376,...,0.005599,0.005563,0.005503,0.005465,0.005341,0.005305,0.005272,0.005242,0.005206,0.005167
Algeria: Tebboune,0.993810,0.979048,0.973364,0.958141,0.947408,0.933675,0.924000,0.914612,0.908370,0.898020,...,0.105963,0.105271,0.104574,0.103860,0.103159,0.102470,0.101829,0.101255,0.100558,0.099937
Andorra: Espot Zamora,0.985801,0.978421,0.970158,0.956953,0.946031,0.931912,0.926487,0.919690,0.910704,0.902226,...,0.023392,0.023238,0.023084,0.022927,0.022772,0.022620,0.022479,0.022352,0.022198,0.022024
Angola: Lourenco,0.975911,0.921862,0.909518,0.905946,0.897450,0.884904,0.878615,0.869160,0.861654,0.853856,...,0.059059,0.058673,0.058286,0.057889,0.055939,0.055566,0.055218,0.054907,0.054529,0.054186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela: Nicolas Maduro,0.997543,0.993290,0.990913,0.988686,0.984530,0.981881,0.978523,0.973407,0.939389,0.932764,...,0.083788,0.083239,0.082688,0.080565,0.078729,0.078215,0.077726,0.077287,0.076755,0.076282
Vietnam: Phu Trong,0.983186,0.976467,0.971653,0.818323,0.811781,0.795067,0.789577,0.780696,0.724950,0.720005,...,0.045882,0.045570,0.045259,0.044951,0.044647,0.044349,0.042229,0.041921,0.041632,0.041371
Yemen: Houthi,0.998209,0.983337,0.974691,0.966096,0.958313,0.946798,0.937605,0.927932,0.911492,0.901282,...,0.117645,0.116873,0.116099,0.115307,0.114528,0.113764,0.110606,0.109982,0.109242,0.108583


We won't know how accurate these forecasts are until the future reveals itself to us. However, we can *pretend* to not have the most recent, say, 36 months of data, train a model, then evaluate that model's forecasts over those 36 months. Pretending to not have the most recent 36 months of data is as easy as specifying the TEST_INTERVALS configuration parameter and re-computing leadership spells.

In [9]:
%%capture

TEST_INTERVALS = 36
data_processor.config["TEST_INTERVALS"] = TEST_INTERVALS
data_processor.build_reserved_cols()

modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(n_intervals=TEST_INTERVALS)

We want to evaluate our model on the set of leaders in July 2017, 36 months prior to the most recent month in our dataset. Doing so is as simple as `modeler.evaluate()`. For each time horizon ("Lead Length") we obtain a variety of binary classification metrics, including the [area under the receiver operating characteristic curve (AUROC)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve). Our AUROC values hover around the low-to-mid 0.80's over the first 9 months into the future and around the low-to-mid 0.70's for longer time horizons. The [concordance index ("C-Index")](https://statisticaloddsandends.wordpress.com/2019/10/26/what-is-harrells-c-index/) provides a metric for model performance over all time horizons. Like AUROC, a C-index of 0.5 indicates a useless model, while a C-index of 1.0 is perfect.

In [10]:
metrics = modeler.evaluate()
metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.844560,0.990449,0.984694,193,0,3,0,,0.7009708737864078
2,0.924007,0.980002,0.984694,192,1,3,0,,
3,0.790576,0.969467,0.974490,191,0,4,1,,
4,0.837491,0.949735,0.964286,188,1,5,2,,
5,0.788496,0.933758,0.938776,183,1,10,2,,
6,0.821036,0.917104,0.928571,181,1,10,4,,
7,0.834254,0.899760,0.923469,179,2,10,5,,
8,0.832402,0.882514,0.913265,176,3,11,6,,
9,0.802177,0.866117,0.892857,171,4,14,7,,


The model we trained used default values for hyperparameters such as the maximum number of leaves per tree and the minimum number of observations in each leaf. There are likely other hyperparameter values that would yield better performance for each future period. Searching for those values and returning the best candidate set for each future period (according to performance on a validation set) is as simple as `modeler.hyperoptimize()`. The number of sets to consider, `n_trials`, defaults to 64, but we'll specify a value of 4 to accelerate this example. You can print 'params' after the following chunk to see the iterative steps of parameter identification; however, the output below will display the progress of the function as it attempts to identify optimal values.

In [11]:
N_TRIALS = 4
params = modeler.hyperoptimize(n_trials=N_TRIALS)

[I 2020-09-08 12:28:35,516] Trial 0 finished with value: 0.06059920447311732 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.06059920447311732.
[I 2020-09-08 12:28:35,838] Trial 1 finished with value: 0.06054656727625284 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

[I 2020-09-08 12:28:52,409] Trial 2 finished with value: 0.06807968284425907 and parameters: {'num_iterations': 91, 'learning_rate': 0.08081207670528569, 'num_leaves': 45, 'max_depth': 30, 'min_data_in_leaf': 52, 'min_sum_hessian_in_leaf': 0.14564408551350866, 'bagging_freq': 1, 'bagging_fraction': 0.7421054227078454, 'feature_fraction': 0.9553626362706973, 'lambda_l1': 33.48323382484937, 'lambda_l2': 25.91037545717554, 'min_gain_to_split': 0.039619274608901967, 'min_data_per_group': 284, 'max_cat_threshold': 377, 'cat_l2': 11.820566283956346, 'cat_smooth': 119.75451760945361, 'max_cat_to_onehot': 11, 'max_bin': 150, 'min_data_in_bin': 10}. Best is trial 2 with value: 0.06807968284425907.
[I 2020-09-08 12:28:52,673] Trial 3 finished with value: 0.06611982266579612 and parameters: {'num_iterations': 98, 'learning_rate': 0.48221228650561615, 'num_leaves': 222, 'max_depth': 14, 'min_data_in_leaf': 343, 'min_sum_hessian_in_leaf': 0.16336286068921954, 'bagging_freq': 0, 'bagging_fraction': 

[I 2020-09-08 12:29:10,783] Trial 0 finished with value: 0.07261416310626068 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.07261416310626068.
[I 2020-09-08 12:29:11,109] Trial 1 finished with value: 0.07193871273728797 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

[I 2020-09-08 12:29:24,248] Trial 2 finished with value: 0.06830556165267322 and parameters: {'num_iterations': 91, 'learning_rate': 0.08081207670528569, 'num_leaves': 45, 'max_depth': 30, 'min_data_in_leaf': 52, 'min_sum_hessian_in_leaf': 0.14564408551350866, 'bagging_freq': 1, 'bagging_fraction': 0.7421054227078454, 'feature_fraction': 0.9553626362706973, 'lambda_l1': 33.48323382484937, 'lambda_l2': 25.91037545717554, 'min_gain_to_split': 0.039619274608901967, 'min_data_per_group': 284, 'max_cat_threshold': 377, 'cat_l2': 11.820566283956346, 'cat_smooth': 119.75451760945361, 'max_cat_to_onehot': 11, 'max_bin': 150, 'min_data_in_bin': 10}. Best is trial 2 with value: 0.06830556165267322.
[I 2020-09-08 12:29:24,454] Trial 3 finished with value: 0.06745315991812584 and parameters: {'num_iterations': 98, 'learning_rate': 0.48221228650561615, 'num_leaves': 222, 'max_depth': 14, 'min_data_in_leaf': 343, 'min_sum_hessian_in_leaf': 0.16336286068921954, 'bagging_freq': 0, 'bagging_fraction': 

[I 2020-09-08 12:29:37,560] Trial 0 finished with value: 0.07007999761353152 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.07007999761353152.
[I 2020-09-08 12:29:37,769] Trial 1 finished with value: 0.06777324250741855 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

[I 2020-09-08 12:29:48,942] Trial 2 finished with value: 0.06919407506151465 and parameters: {'num_iterations': 91, 'learning_rate': 0.08081207670528569, 'num_leaves': 45, 'max_depth': 30, 'min_data_in_leaf': 52, 'min_sum_hessian_in_leaf': 0.14564408551350866, 'bagging_freq': 1, 'bagging_fraction': 0.7421054227078454, 'feature_fraction': 0.9553626362706973, 'lambda_l1': 33.48323382484937, 'lambda_l2': 25.91037545717554, 'min_gain_to_split': 0.039619274608901967, 'min_data_per_group': 284, 'max_cat_threshold': 377, 'cat_l2': 11.820566283956346, 'cat_smooth': 119.75451760945361, 'max_cat_to_onehot': 11, 'max_bin': 150, 'min_data_in_bin': 10}. Best is trial 1 with value: 0.0677954253121009.
[I 2020-09-08 12:29:49,142] Trial 3 finished with value: 0.06838808742064523 and parameters: {'num_iterations': 98, 'learning_rate': 0.48221228650561615, 'num_leaves': 222, 'max_depth': 14, 'min_data_in_leaf': 343, 'min_sum_hessian_in_leaf': 0.16336286068921954, 'bagging_freq': 0, 'bagging_fraction': 0

[I 2020-09-08 12:30:00,304] Trial 0 finished with value: 0.07009298240867941 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.07009298240867941.
[I 2020-09-08 12:30:00,486] Trial 1 finished with value: 0.06806425742289356 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

Using our new hyperparameter sets is as simple as passing them to build_model. Because the number of trees to train was one of our hyperparameters, we don't use early stopping and don't need a validation set.

In [12]:
modeler.build_model(n_intervals=TEST_INTERVALS, params=params)

We can now evaluate our new model and directly compare it to our default model. The hyperparameters that performed best on a validation set don't always outperform the default parameters on the test set, but in this case they do. We see that our new hyperparameters yield higher AUROCs and a higher C-Index than the defaults.

In [13]:
hyperoptimized_metrics = modeler.evaluate()
hyperoptimized_metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.915371,0.985451,0.984694,193,0,3,0,,0.7261326860841424
2,0.899827,0.975741,0.984694,192,1,3,0,,
3,0.806283,0.964537,0.974490,191,0,4,1,,
4,0.858655,0.944886,0.964286,189,0,5,2,,
5,0.778080,0.923021,0.938776,183,1,8,4,,
6,0.822214,0.902469,0.928571,181,1,9,5,,
7,0.843094,0.884635,0.923469,179,2,7,8,,
8,0.832074,0.871631,0.913265,176,3,9,8,,
9,0.844626,0.856248,0.892857,170,5,10,11,,


To use our new hyperparameters to get forecasts, we re-train our model with the most recent 36 months included. However, because we only obtained hyperparameters for time horizons up through 36 months, we can only forecast that far ahead.

In [14]:
data_processor.config["TEST_INTERVALS"] = 0
data_processor.build_reserved_cols()
modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(n_intervals=TEST_INTERVALS, params=params)

Obtaining forecasts is as easy as before.

In [15]:
forecasts = modeler.forecast()
forecasts.columns = custom_forecast_headers[:len(forecasts.columns)]
forecasts

,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,...,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Feb 2023,Mar 2023,Apr 2023,May 2023,Jun 2023,Jul 2023
country-leader,,,,,,,,,,,,,,,,,,,,,
Afghanistan: Ashraf Ghani,0.991093,0.979848,0.957265,0.949273,0.941592,0.930601,0.921169,0.901205,0.895184,0.888945,...,0.736697,0.727445,0.718451,0.709210,0.706980,0.698771,0.691171,0.682120,0.672419,0.660516
Albania: Rama,0.996797,0.990388,0.982417,0.979885,0.966117,0.937893,0.933570,0.928595,0.924471,0.902901,...,0.482605,0.472994,0.464972,0.459507,0.455262,0.445264,0.442457,0.437688,0.432696,0.428692
Algeria: Tebboune,0.990184,0.984362,0.972563,0.956857,0.951445,0.944043,0.936443,0.932983,0.929100,0.925302,...,0.775578,0.766530,0.756751,0.747865,0.741983,0.735685,0.732794,0.724444,0.717236,0.714697
Andorra: Espot Zamora,0.993210,0.986665,0.976904,0.973907,0.968199,0.957688,0.952992,0.948905,0.934777,0.927245,...,0.705549,0.694359,0.683386,0.673157,0.666243,0.661205,0.657431,0.644515,0.541050,0.526654
Angola: Lourenco,0.997097,0.996140,0.989850,0.987106,0.984528,0.981400,0.979852,0.977705,0.976321,0.974800,...,0.852196,0.842822,0.835923,0.828924,0.824795,0.819852,0.815952,0.807849,0.801043,0.798222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela: Nicolas Maduro,0.996305,0.994335,0.992993,0.991296,0.987262,0.985160,0.984604,0.981730,0.971248,0.970000,...,0.815916,0.805015,0.792117,0.779197,0.762821,0.740843,0.473810,0.467361,0.460709,0.457674
Vietnam: Phu Trong,0.986478,0.981187,0.971295,0.963948,0.960982,0.946975,0.867683,0.851582,0.837019,0.833456,...,0.579518,0.572372,0.564847,0.557877,0.538515,0.532735,0.529316,0.522721,0.515941,0.512296
Yemen: Houthi,0.992535,0.990444,0.981565,0.969736,0.966379,0.954898,0.945477,0.893588,0.890401,0.887513,...,0.742775,0.732881,0.721961,0.710819,0.697256,0.684700,0.677049,0.668970,0.659283,0.656526
